## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Customer Orders Data Generation

### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Pip Installs

In [ ]:
# To read/write to/from Kafka
import sys

# https://PLACEHOLDER.com/index.html
#!{sys.executable} -m pip install PLACEHOLDER

### Initialize

In [ ]:
from PIL import Image
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "us"
region = "us-central1"
location = "us-central1"
storage_account = "data-analytics-preview"
public_storage_storage_account = "data-analytics-golden-demo"
table_name = "order"
dataset_name = "chocolate_ai"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### Helper Methods

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Gemini LLM - Multimodal

In [ ]:
def GeminiLLM_Multimodal(multimodal_prompt_list, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": multimodal_prompt_list
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Imagen3 Image Generation

In [ ]:
def ImageGen(prompt):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  model_version = "imagen-3.0-generate-001" # imagen-3.0-fast-generate-001
  #model_version = "imagen-3.0-generate-preview-0611" # Preview Access Model

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-generation
  # url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/imagegeneration:predict"
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/{model_version}:predict"

  payload = {
    "instances": [
      {
        "prompt": prompt
      }
    ],
    "parameters": {
      "sampleCount": 1,
      "personGeneration" : "allow_adult"  # Google users will get an error due to an org policy
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    response_json = json.loads(response.content)
    # print(f"Imagen3 response_json: {response_json}")

    if "blocked" in response_json:
      print(f"Blocked: {response_json['blocked']}")

    if "predictions" in response_json:
      image_data = response_json["predictions"][0]["bytesBase64Encoded"]
      image_data = base64.b64decode(image_data)
      filename= str(uuid.uuid4()) + ".png"
      with open(filename, "wb") as f:
        f.write(image_data)
      print(f"Image generated OK.")
      return filename
    else:
      raise RuntimeError(f"No predictions in response: {response.content}")
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Helper Functions

In [ ]:
def RunQuery(sql):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      raise Exception(query_job.error_result)

In [ ]:
def GetTableSchema(project_id, dataset_name, table_name):
  import io
  from google.cloud import bigquery

  client = bigquery.Client()

  dataset_ref = client.dataset(dataset_name, project=project_id)
  table_ref = dataset_ref.table(table_name)
  table = client.get_table(table_ref)

  f = io.StringIO("")
  client.schema_to_json(table.schema, f)
  return f.getvalue()

In [ ]:
def GetDistinctValues(project_id, dataset_name, table_name, field_name):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = f"""
  SELECT STRING_AGG(DISTINCT {field_name}, "," ) AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """

  df_result = client.query(sql).to_dataframe()
  result_str = df_result['result'].iloc[0]
  if result_str is None:
    return ""
  else:
    return result_str

In [ ]:
def GetStartingValue(project_id, dataset_name, table_name, field_name):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) + 1 AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """

  df_result = client.query(sql).to_dataframe()
  return df_result['result'].iloc[0]

In [ ]:
def GetMaxValue(project_id, dataset_name, table_name, field_name):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """

  df_result = client.query(sql).to_dataframe()
  return df_result['result'].iloc[0]

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  #print(json_formatted_str)
  return json_formatted_str

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

### BigQuery Tables

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE `data-analytics-preview.chocolate_ai.store`
(
    store_id INT64 NOT NULL OPTIONS(description="Primary key."),
    store_name STRING NOT NULL OPTIONS(description="Name of the store"),
    store_latitude FLOAT64 NOT NULL OPTIONS(description="Latitude of the store"),
    store_longitude FLOAT64 NOT NULL OPTIONS(description="Longitude of the store"),
)
CLUSTER BY store_id;

INSERT INTO `data-analytics-preview.chocolate_ai.store`
(store_id, store_name, store_latitude, store_longitude)
VALUES
(1, 'Arc de Triomphe', 48.87397729997744, 2.2948883077615134),
(2, 'Eiffel Tower', 48.858409241104404, 2.2945373267867164),
(3, 'Louvre Museum', 48.860475868131616, 2.338965431077322);

In [ ]:
%%bigquery

CREATE TABLE IF NOT EXISTS `data-analytics-preview.chocolate_ai.order`
(
    order_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    store_id INTEGER NOT NULL OPTIONS(description="Foreign key: store table."),
    customer_id INTEGER NOT NULL OPTIONS(description="Foreign key: Customer table."),
    order_datetime TIMESTAMP NOT NULL OPTIONS(description="The datetime the order was started."),
    order_completion_datetime TIMESTAMP NOT NULL OPTIONS(description="The datetime the order was completed.")
)
CLUSTER BY order_id, store_id;


CREATE TABLE IF NOT EXISTS `data-analytics-preview.chocolate_ai.order_item`
(
    order_item_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    order_id INTEGER NOT NULL OPTIONS(description="Foreign key: Order table"),
    menu_id INTEGER NOT NULL OPTIONS(description="Foreign key: Menu table"),
    quantity INTEGER NOT NULL OPTIONS(description="Number of items ordered"),
    item_size STRING NOT NULL OPTIONS(description="Size of the item"),
    item_price FLOAT64 NOT NULL OPTIONS(description="Price of the item"),
    item_total FLOAT64 NOT NULL OPTIONS(description="Total price of the item")
)
CLUSTER BY order_id;

In [ ]:
def GetMenuItems():
  sql = f"""SELECT TO_JSON(STRUCT(menu_name, menu_description)) AS menu_item_json
  FROM `data-analytics-preview.chocolate_ai.menu`"""

  result_df = RunQuery(sql)
  result_str = ""

  for index, row in result_df.iterrows():
    result_str = result_str + row['menu_item_json'] + ","

  return '[' + result_str + ']'

### Generate Orders

In [ ]:
max_store_id = GetMaxValue('data-analytics-preview', dataset_name, "store", "store_id")
max_customer_id = GetMaxValue('data-analytics-preview', dataset_name, "customer", "customer_id")
max_menu_id = GetMaxValue('data-analytics-preview', dataset_name, "menu", "menu_id")

In [ ]:
for i in range(1,2):

  sql = f"""INSERT INTO
            `data-analytics-preview.chocolate_ai.order` (
              order_id,
              store_id,
              customer_id,
              order_datetime,
              order_completion_datetime)

          WITH
            data_max_id AS (
              SELECT IFNULL(MIN(order_datetime),CAST('2020-01-01' AS TIMESTAMP)) AS min_order_datetime,
                    IFNULL(MAX(order_datetime),CAST('2020-01-01' AS TIMESTAMP)) AS max_order_datetime,
                    IFNULL(MAX(order_id),0) AS max_id,
                    COUNT(*) AS record_count
                FROM`data-analytics-preview.chocolate_ai.order`
            ),

            data_random_data AS (
              SELECT CAST(ROUND(1 + RAND() * ({max_store_id}     - 1)) AS INT64) AS store_id,
                    CAST(ROUND(1 + RAND() * ({max_customer_id} - 1)) AS INT64) AS customer_id,
                    TIMESTAMP_ADD(data_max_id.max_order_datetime, INTERVAL CAST(ROUND(1 + RAND() * (30 * 24 * 60 * 74) - 1) AS INT64) MINUTE) AS order_datetime, -- 74 months of minutes
                FROM UNNEST(GENERATE_ARRAY(1, 1000000)) AS element
                    CROSS JOIN data_max_id
            ),

            data_random_all_data AS (
              SELECT *,
                    TIMESTAMP_ADD(order_datetime, INTERVAL CAST(ROUND(60 + RAND() * ((60*15)) - 60) AS INT64) SECOND) AS order_completion_datetime, -- from 60 seconds to 15 minutes
                FROM data_random_data
            ),

            data_ranked AS (
              SELECT *,
                    ROW_NUMBER() OVER (ORDER BY order_datetime) AS row_number
              FROM data_random_all_data
            )
          SELECT row_number +  max_id AS  order_id,
              store_id,
              customer_id,
              order_datetime,
              order_completion_datetime
          FROM data_ranked
          CROSS JOIN data_max_id;
  """

  RunQuery(sql)

  print(f"Order Loop {i} generated")


In [ ]:
%%bigquery

SELECT MIN(order_datetime),
       MAX(order_completion_datetime)
 FROM `data-analytics-preview.chocolate_ai.order`;

In [ ]:
# Create the order items

sql = f"""INSERT INTO `data-analytics-preview.chocolate_ai.order_item` (order_item_id, order_id, menu_id, quantity, item_size, item_price, item_total)
          WITH
            data_max_id AS (
              SELECT IFNULL(MAX(order_item_id),0) AS max_id
                FROM `data-analytics-preview.chocolate_ai.order_item`
            ),

            -- order's without items
            data_order AS (
              SELECT order_id,
                    CASE WHEN RAND() <= .5 THEN 1 -- most order are 1 item
                          ELSE CAST(ROUND(2 + RAND() * (5 - 2)) AS INT64) -- up to 5 items per order
                      END AS order_item_count
                FROM `data-analytics-preview.chocolate_ai.order` AS order_t
              WHERE NOT EXISTS (SELECT * FROM `data-analytics-preview.chocolate_ai.order_item` AS order_item WHERE order_t.order_id = order_item.order_id)
            ),

            data_order_with_array AS
            (
              select order_id,
                    order_item_count,
                    GENERATE_ARRAY(1, order_item_count) as order_item_array,
                    RAND() AS quantity_rand
                from data_order
            ),

            data_random_all_data AS (
              SELECT order_id,
                    order_item AS order_item_id,
                    CAST(ROUND(1 + RAND() * (250 - 1)) AS INT64) AS menu_id,
                    CASE WHEN quantity_rand <= .75 THEN 1 -- most orders are 1 quanity
                        WHEN quantity_rand <= .85 THEN 2
                        WHEN quantity_rand <= .90 THEN 3
                        WHEN quantity_rand <= .95 THEN 4
                        ELSE 5
                      END AS quantity,
                from data_order_with_array
                    CROSS JOIN UNNEST(order_item_array) AS order_item
            ),

            data_ranked AS (
              SELECT *,
                    ROW_NUMBER() OVER (ORDER BY order_id, order_item_id) AS row_number
              FROM data_random_all_data
            )
          SELECT row_number +  max_id AS order_item_id,
                data_ranked.order_id,
                data_ranked.menu_id,
                data_ranked.quantity,
                menu.menu_size,
                menu.menu_price,
                ROUND(CAST(data_ranked.quantity * menu.menu_price AS NUMERIC), 2, "ROUND_HALF_EVEN") AS item_total
            FROM data_ranked
                CROSS JOIN data_max_id
                INNER JOIN `data-analytics-preview.chocolate_ai.menu` AS menu
                        ON data_ranked.menu_id = menu.menu_id;"""


RunQuery(sql)


In [ ]:
%%bigquery

SELECT COUNT(*) AS cnt
  FROM `data-analytics-preview.chocolate_ai.order` AS parent
WHERE NOT EXISTS (SELECT 1 FROM `data-analytics-preview.chocolate_ai.order_item` AS child WHERE parent.order_id = child.order_id);

In [ ]:
%%bigquery

SELECT avg(item_total)
  from  `data-analytics-preview.chocolate_ai.order` as parent
      inner join `data-analytics-preview.chocolate_ai.order_item` as child on parent.order_id = child.order_id

### Verify Orders Items

### Clean Up

In [ ]:
# Placeholder

### Reference Links


- [Google.com](https://www.google.com)